# Command

> Converting instructions into object representations

In this scenario we're going to imagine that we're creating a bank's IT systems. Let's implement a class to represent a bank account: the account will contain a balance that we can deposit money to or withdraw from, and we will set an overdraft limit in order to avoid going into deep negative balance.

(For simplicity's sake, we'll obviate basic security stuff like input sanitation).

In [12]:
class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')

    def __str__(self):
        return f'Balance = {self.balance}'

This code works fine but we're missing a crucial aspect of any real bank: any transaction must be recorded. Any deposits or withdrawals that we do on our class will simply happen and will not leave any trace of them happening at all.

By using the **Command Design Pattern**, instead of calling our `withdraw` and `deposit` methods we can provide an interface instead that will create a record of any and all calls.

We'll begin by creating a `Command` base class that will define our basic interface:

In [13]:
from abc import ABC

class Command(ABC):
    def invoke(self):
        pass

> In this scenario the base class isn't strictly necessary because we're only implementing one command class, but it's good practice.

Now let's implement our actual `BankAccountCommand`. In order to perform actions on a bank account, we need to know which bank account we need to perform the action on, the action itself (which will be either `deposit` or `withdraw`) and any arguments for the action, which for our case is the amount of money. For the actions, we'll use simple enums to identify them.

In [14]:
from enum import Enum

class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.amount = amount
        self.action = action
        self.account = account

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            # record the operation here!
            self.account.deposit(self.amount)
        elif self.action == self.Action.WITHDRAW:
            # record the operation here!
            self.account.withdraw(self.amount)

> Once again for simplicity's sake we'll omit the action recording step, but you would implement it when calling the actual `BankAccount` method.

Let's see our code in action:

In [15]:
ba = BankAccount()
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print('After $100 deposit:', ba)

Deposited 100, balance = 100
After $100 deposit: Balance = 100


A bonus side effect of the Command Design Pattern is that we can easily implement undo operations right inside the command.

We will implement a new interface member called `undo` which will roll back the change we've made: we will simply perform the opposite action of the `BankAccountCommand` instance (this isn't the proper way of doing things in a bank, but it will do for this demo).

In [16]:
class Command(ABC):
    def invoke(self):
        pass

    def undo(self): # new interface member
        pass

class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.amount = amount
        self.action = action
        self.account = account

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.withdraw(self.amount)

    def undo(self):
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

Let's see it in action:

In [17]:
ba = BankAccount()
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print('After $100 deposit:', ba)

cmd.undo()
print('$100 deposit undone:', ba)

Deposited 100, balance = 100
After $100 deposit: Balance = 100
Withdrew 100, balance = 0
$100 deposit undone: Balance = 0


Pretty cool, but our implementation has a fatal flaw: we are not keeping track of whether a command has succeeded or not, and this can result in unexpected side effects when trying to undo the operations. Let's see an example:

In [18]:
ba = BankAccount()
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print('After $100 deposit:', ba)

cmd.undo()
print('$100 deposit undone:', ba)

illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 1000)
illegal_cmd.invoke()
print('After impossible withdrawal:', ba)
illegal_cmd.undo()
print('After undo:', ba)

Deposited 100, balance = 100
After $100 deposit: Balance = 100
Withdrew 100, balance = 0
$100 deposit undone: Balance = 0
After impossible withdrawal: Balance = 0
Deposited 1000, balance = 1000
After undo: Balance = 1000


Oopsie whoopsie, we undid a withdrawal that didn't happen at all because it exceeded the overdraft limit, and now this bank account magically has $1000. Great for the end customer but not for the bank.

We need to track whether or not a particular operation has succeeded. We can do this by adding a success flag on our command that will be `True` if the operation succeeds or `False` otherwise.

For `deposit`, we can confidently set the flag to `True` after invoking the command, but for `withdraw` we will need to modify the original `BankAccount` in order to have it return whether or not it succeeded.

In [19]:
class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            return True # The withdrawal succeeded
        return False # Withdrawal cancelled because it exceeded the overdraft limit

    def __str__(self):
        return f'Balance = {self.balance}'
    
class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.amount = amount
        self.action = action
        self.account = account
        self.success = None # success flag

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True # Deposits always work
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount) # We check the result of the withdraw action to determine whether or not it succeeded

    def undo(self):
        if not self.success:
            return # If the action did not succeed, return without doing anything else
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

Let's see if our changes stop the illegal operation from happening:

In [20]:
ba = BankAccount()
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print('After $100 deposit:', ba)

cmd.undo()
print('$100 deposit undone:', ba)

illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 1000)
illegal_cmd.invoke()
print('After impossible withdrawal:', ba)
illegal_cmd.undo()
print('After undo:', ba)

Deposited 100, balance = 100
After $100 deposit: Balance = 100
Withdrew 100, balance = 0
$100 deposit undone: Balance = 0
After impossible withdrawal: Balance = 0
After undo: Balance = 0


Success! The undo trick cannot be exploited anymore.

Note that some side effects still can happen with our current code: we can still undo a single operation multiple times; we can fix this by setting `success` to `None` after undoing, but that would limit undoing to a single step. We could then set `self.action` to its opposite within `undo`, but that would cycle between opposing actions whenever we would call `undo`.

The optimal solution to this is keeping a log of our actions (the step we omitted doing for simplicity): whenever we invoke successfully, we append the action to a list within our command instance, and whenever we undo, we pop the last successful action from the list and perform the opposite action. This approach is called **Event Sourcing**.

Let's modify our `BankAccountCommand` to include a log and see how it would perform:

In [21]:
class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.account = account
        self.amount = amount
        self.action = action
        # we don't need the success flag anymore
        self.logs = [] # we add the logs

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            success = True
        elif self.action == self.Action.WITHDRAW:
            success = self.account.withdraw(self.amount)

        # Add to logs
        if success:
            self.logs.append(self.action)

    def undo(self):
        if not self.logs: # We no longer have to rely on success
            return
        most_recent_user_action = self.logs.pop()
        if most_recent_user_action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)
            self.action = self.Action.DEPOSIT
        elif most_recent_user_action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
            self.action = self.Action.WITHDRAW

In [22]:
ba = BankAccount()
print("> Init")
print(ba)

print("> Add +100")
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print(ba)

print("> Undo")
cmd.undo()
print(ba)

# Fixed broken undo
print("> Fixed: Undo once again")
cmd.undo()
print(ba)

# But now we cycle thorugh
print("> Wee! We have the entire history of you")
cmd.undo()
print(ba)

# Even if we go too far
print("> We don't go beyond what we know")
cmd.undo()
print(ba)

print("> Withdraw 500")
illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 500)
illegal_cmd.invoke()
print(ba)

print("> Withdraw too much")
illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 5000)
illegal_cmd.invoke()
print(ba)

> Init
Balance = 0
> Add +100
Deposited 100, balance = 100
Balance = 100
> Undo
Withdrew 100, balance = 0
Balance = 0
> Fixed: Undo once again
Balance = 0
> Wee! We have the entire history of you
Balance = 0
> We don't go beyond what we know
Balance = 0
> Withdraw 500
Withdrew 500, balance = -500
Balance = -500
> Withdraw too much
Balance = -500


(Thanks to [Niyas Mohammed](https://github.com/niazangels) for [his code snippet](https://gist.github.com/niazangels/a90778ae7f94501111144c7cbba72d63))